In [81]:
using Distributions
using Rocket
using ReactiveMP
using BenchmarkTools

using Profile
using PProf
using ProfileSVG

import Base: show

In [88]:
function kalman_filter_graph()
    x_prev_add = AdditionNode()
    
    x_prior = datavar(:x_prior, Normal{Float64})
    add_1   = constvar(:add_1, 1.0)

    connect!(x_prev_add, :in1, x_prior, 1)
    connect!(x_prev_add, :in2, add_1, 1)
    
    noise = constvar(:noise, Normal(0.0, sqrt(200.0)))
    
    add_x_and_noise = AdditionNode()
    
    x = simplerandomvar(:x)
    
    connect!(x_prev_add, :out, x, 1)
    connect!(add_x_and_noise, :in1, x, 2)
    connect!(add_x_and_noise, :in2, noise, 1)
    
    y = datavar(:y, Float64)
    
    connect!(add_x_and_noise, :out, y, 1)
    
    activate!(x_prev_add)
    activate!(add_x_and_noise)
    
    return x_prior, x, y
end

function kalman(data)
    N = length(data)
    
    x_prior, x, y = kalman_filter_graph()

    link(x_prior, getmarginal(x))
    update!(x_prior, Normal(0.0, sqrt(10000.0)))
    
    beliefs = Vector{Normal{Float64}}(undef, N)
    
    subscribe!(getmarginal(x) |> enumerate(), (t) -> beliefs[t[1]] = getdata(t[2]))
    
    for (i, d) in enumerate(data)
        update!(y, d)
    end
    
    return beliefs
end

kalman (generic function with 1 method)

In [89]:
N = 600
data = collect(1:N) + sqrt(200.0) * randn(N);

In [92]:
@btime kalman($data)

  985.872 μs (16172 allocations: 697.88 KiB)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=-10.7605049574181, σ=14.002800840280099)
 Normal{Float64}(μ=-8.349458173498022, σ=9.950371902099892)
 Normal{Float64}(μ=2.4369115283480167, σ=8.137884587711596)
 Normal{Float64}(μ=1.7891365568295168, σ=7.053456158585984)
 Normal{Float64}(μ=3.6456352608784295, σ=6.311944030978033)
 Normal{Float64}(μ=7.013040971330439, σ=5.763904177042351)
 Normal{Float64}(μ=6.860314760295695, σ=5.337605126836238)
 Normal{Float64}(μ=7.975502357321227, σ=4.993761694389224)
 Normal{Float64}(μ=8.961751642333008, σ=4.708816093480111)
 Normal{Float64}(μ=11.693527641875335, σ=4.467670516087703)
 Normal{Float64}(μ=11.043707439626392, σ=4.2601432284230505)
 Normal{Float64}(μ=13.325892787649707, σ=4.079085082240021)
 Normal{Float64}(μ=15.13684377969131, σ=3.919309008348103)
 ⋮
 Normal{Float64}(μ=588.0708575643617, σ=0.582706653680032)
 Normal{Float64}(μ=589.0910160316671, σ=0.5822126418123303)
 Normal{Float64}(μ=590.0798968277409, σ=0.5817198842703781)
 Nor

In [91]:
@time kalman(data)

  0.001596 seconds (16.17 k allocations: 697.875 KiB)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=-10.7605049574181, σ=14.002800840280099)
 Normal{Float64}(μ=-8.349458173498022, σ=9.950371902099892)
 Normal{Float64}(μ=2.4369115283480167, σ=8.137884587711596)
 Normal{Float64}(μ=1.7891365568295168, σ=7.053456158585984)
 Normal{Float64}(μ=3.6456352608784295, σ=6.311944030978033)
 Normal{Float64}(μ=7.013040971330439, σ=5.763904177042351)
 Normal{Float64}(μ=6.860314760295695, σ=5.337605126836238)
 Normal{Float64}(μ=7.975502357321227, σ=4.993761694389224)
 Normal{Float64}(μ=8.961751642333008, σ=4.708816093480111)
 Normal{Float64}(μ=11.693527641875335, σ=4.467670516087703)
 Normal{Float64}(μ=11.043707439626392, σ=4.2601432284230505)
 Normal{Float64}(μ=13.325892787649707, σ=4.079085082240021)
 Normal{Float64}(μ=15.13684377969131, σ=3.919309008348103)
 ⋮
 Normal{Float64}(μ=588.0708575643617, σ=0.582706653680032)
 Normal{Float64}(μ=589.0910160316671, σ=0.5822126418123303)
 Normal{Float64}(μ=590.0798968277409, σ=0.5817198842703781)
 Nor

In [90]:
@time kalman(data)

  0.187965 seconds (496.15 k allocations: 25.588 MiB)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=-10.7605049574181, σ=14.002800840280099)
 Normal{Float64}(μ=-8.349458173498022, σ=9.950371902099892)
 Normal{Float64}(μ=2.4369115283480167, σ=8.137884587711596)
 Normal{Float64}(μ=1.7891365568295168, σ=7.053456158585984)
 Normal{Float64}(μ=3.6456352608784295, σ=6.311944030978033)
 Normal{Float64}(μ=7.013040971330439, σ=5.763904177042351)
 Normal{Float64}(μ=6.860314760295695, σ=5.337605126836238)
 Normal{Float64}(μ=7.975502357321227, σ=4.993761694389224)
 Normal{Float64}(μ=8.961751642333008, σ=4.708816093480111)
 Normal{Float64}(μ=11.693527641875335, σ=4.467670516087703)
 Normal{Float64}(μ=11.043707439626392, σ=4.2601432284230505)
 Normal{Float64}(μ=13.325892787649707, σ=4.079085082240021)
 Normal{Float64}(μ=15.13684377969131, σ=3.919309008348103)
 ⋮
 Normal{Float64}(μ=588.0708575643617, σ=0.582706653680032)
 Normal{Float64}(μ=589.0910160316671, σ=0.5822126418123303)
 Normal{Float64}(μ=590.0798968277409, σ=0.5817198842703781)
 Nor